In [12]:
## reviews sample
import pandas as pd
import gzip
import json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        if i<=1000:
            df[i] = d
            i += 1
        else:
            return pd.DataFrame.from_dict(df, orient='index')
            break
    

review = getDF('D:/HKUST/Research/ChatGPT/Books_5.json.gz')

In [2]:
review=df['reviewText']
review[0]

"The King, the Mice and the Cheese by Nancy Gurney is an excellent children's book.  It is one that I well remember from my own childhood and purchased for my daughter who loves it.\n\nIt is about a king who has trouble with rude mice eating his cheese. He consults his wise men and they suggest cats to chase away the mice. The cats become a nuisance, so the wise men recommend the king bring in dogs to chase the cats away.  The cycle goes on until the mice are finally brought back to chase away the elephants, brought in to chase away the lions that'd chased away the dogs.\n\nThe story ends in compromise and friendship between the mice and the king.  The story also teaches cause and effect relationships.\n\nThe pictures that accompany the story are humorous and memorable.  I was thrilled to discover that it is back in print.  I *highly* recommend it for children ages 2 to 7."

In [13]:
## ChatGPT API
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [14]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [15]:
## User views and clicks data
Clicks_views=[]
#clicks and views should also be product name/URL.It might be embeded into the consumers' responses.

# process:

### users choice: 1. recommendation; 2. reviews summary

## 1. recommendation
### feed information: 

a. product data; 

b. clicks and views (real time changes) 

c. users responses (real time changes)

could embed the clicks and views into the users responses.

### output: product with URL link.

## 2. reviews summarize
feed information: 
    
a. if in product webpage: grab comments from the present webpage in real time; 

    
b. if not in product webpage: ask the users to give product names/link/open product webpage.

### output: provide summarize information and recommend targeted consumers


In [20]:

# products data should include the product data of the whole online shopping mall. 
products=pd.read_csv("D:/HKUST/Research/ChatGPT/products.csv")
products=products.loc[:,["goods_name","intro","price","selling_point"]]

In [21]:
products

,goods_name,intro,price,selling_point
0,6ft of Acid Free Linen Tape for Bookbinding,"[""Deluxe Duffel Bag with full length zipper de...",4.00,"[""Cotton Canvas"",""Acid Free""]"
1,Scholastic Pirate's Treasure Fundle,"[""Shiver me timbers! Solve I SPY pirate pictur...",2.59,"[""Software: Play I SPY games to uncover the hi..."
2,Putting Set in Case,"[""Create an instant indoors putting green by p...",35.00,[]
3,"Insulated Lunch Bag, Lunchbox, Bento Box (Pack...","[""Not so high deluxe qualify, but really good ...",3.00,"[""Size: Big bag 8.7\""L x 5.7\""H x 5.8\""W Zippe..."
4,"WELLAND Chicago Wall Floating Corner Shelf, 20...","[""Chicago Floating Shelf is an impressively se...",3.00,"[""Dimensions: 20-inch W by 20-inch D by 1.75-i..."
...,...,...,...,...
995,Ironwood Gourmet 11x13 Wood Steak Serving Plate.,"[""For your grilled food, this handsome plate i...",29.99,"[""Material: Acacia Wood"",""Dimensions: 11 x 13 ..."
996,TableCraft 1112KMC 3-Piece Nostalgia Squeeze B...,"[""Old time design style 12-ounce squeeze bottl...",8.97,"[""Diner style"",""12-ounce capacity"",""Contains m..."
997,Tablecraft 6 Piece Assorted Sandwich &amp; Fry...,"[""Sandwich and fry basket. Dishwasher and micr...",9.45,"[""Sandwich and fry basket"",""Dishwasher and mic..."
998,"Oggi 7059 Stainless Steel Utensil Holder, Small","[""Keep your cooking utensils and tools organiz...",12.50,"[""Durable brushed stainless-steel"",""Great for ..."


In [23]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':f"""

You are ShoppingBot, an automated assistant to help consumers find ideal product in an on-line shopping mall. \

Your task is to recommend product in this shopping mall with URL and summarize product reviews. 

You first greet the customer, and ask if the customer wants you to make a recommendation or to summarize the product reviews.

if want you to make a recommendation:

    Ask them what product of category they want.

    These are the products that this online shopping mall have, including the name of the products,introduction, price and selling_point.

    products: ```{products}```
    
    Note that you do not need to show all the products.Just ask them their demands and recommend.

    You will recommend products based on their preference and their previous clicks and views if they have. You should only show less than 5 products.

    Your recommendation should not be beyond what the shopping mall have. And you will give the product URL to customer and explain recommendation reason.

    If the user ask you anything beyond the products, please tell them we don't have this product.
    
if want you to summarize the product reviews:

    summarize the product reviews: ```{review}```
    
    Give a detailed summarize for positive reviews.Please list five of the positive reviews.
    
    The summarize is intended for customers. So please also give a detailed recommendation on what kind of customers is this product suitable for.
    
Then, you ask is there anything you could help. 


 """} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard



Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

### previous version